In [55]:
from math import sqrt
import numpy as np
from numpy import split
from numpy import array
from pandas import read_csv
import pandas as pd
import pickle
from sklearn.metrics import mean_squared_error,mean_absolute_error
from matplotlib import pyplot
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
pd.options.display.float_format='{:,.20}'.format

In [56]:
# split a univariate dataset into train/test sets
def split_dataset(data,n_input):
	# split into standard weeks
	train, test = data[0:168], data[168:174]   
	# restructure into windows of weekly data
	train = array(split(train, len(train)/n_input))
	test = array(split(test, len(test)/n_input))
	return train, test

In [57]:
 
# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts(actual, predicted):
    scores = list()
    
    #print("Printing actual and predicted " , actual[0 , :] , predicted[0,:])
    # calculate an RMSE score for each day
    for i in range(actual.shape[1]):
        # calculate mse
        #print(actual.shape)
        #print(predicted.shape)
        mse = mean_absolute_error(actual[:, i], predicted[:, i])
        # calculate rmse
        rmse = mse
        # store
        scores.append(rmse)
    # calculate overall RMSE
    s = 0
    for row in range(actual.shape[0]):
        for col in range(actual.shape[1]):
            s += np.abs((actual[row,col] - predicted[row,col]))
    score = (s / (actual.shape[0] * actual.shape[1]))
    return score, scores, predicted[0,:]

In [58]:
# summarize scores
def summarize_scores(name, score, scores):
	s_scores = ', '.join(['%.15f' % s for s in scores])
#print('%s: [%.15f] %s' % (name, score, s_scores))
 
# prepare a list of ml models
def get_models(col):
	# linear models
	if(col==14):
		rfr = RandomForestRegressor()
		return rfr
	else:    
		lrr= LinearRegression()
		return lrr
        #print('Defined %d models' % len(models))


In [59]:
# create a feature preparation pipeline for a model
def make_pipeline(model):
	steps = list()
	# standardization
	steps.append(('standardize', StandardScaler()))
	# normalization
	steps.append(('normalize', MinMaxScaler()))
	# the model
	steps.append(('model', model))
	# create pipeline
	pipeline = Pipeline(steps=steps)
	return pipeline

In [60]:
# make a recursive multi-step forecast
def forecast(model, input_x, n_input):
	yhat_sequence = list()
	input_data = [x for x in input_x]
#	print(input_x)
	for j in range(n_input):
		# prepare the input data
		X = array(input_data[-n_input:]).reshape(1, n_input)       
		# make a one-step forecast
		yhat = model.predict(X)[0]
		# add to the result
		yhat_sequence.append(yhat)
		# add the prediction to the input
		input_data.append(yhat)
	return yhat_sequence

In [61]:
# convert windows of weekly multivariate data into a series of total power
def to_series(data,col):
	# extract just the total power from each week
	series = [week[:, col] for week in data]
	# flatten into a single series
	series = array(series).flatten()
	return series

In [62]:
# convert history into inputs and outputs
def to_supervised(history, n_input,col):
	# convert history to a univariate series
	data = to_series(history,col)
	X, y = list(), list()
	ix_start = 0
	# step over the entire history one time step at a time
	for i in range(len(data)):
		# define the end of the input sequence
		ix_end = ix_start + n_input
		# ensure we have enough data for this instance
		if ix_end < len(data):
			X.append(data[ix_start:ix_end])
			y.append(data[ix_end])
		# move along one time step
		ix_start += 1
	return array(X), array(y)

In [67]:
# fit a model and make a forecast
def sklearn_predict(model, history, n_input,col):
	# prepare data
    train_x, train_y = to_supervised(history, n_input,col)
	# make pipeline
    pipeline = make_pipeline(model)
	# fit the model
    pipeline.fit(train_x, train_y)
    #preparing dump file
    #1=str(model)+"_"+str(col)
    str1 =str(col)+"_1.sav"
    filename = str1
    pickle.dump(model, open(filename, 'wb'))
    
	# predict the week, recursively
    yhat_sequence = forecast(pipeline, train_x[-1, :], n_input)
    return yhat_sequence

In [68]:
# evaluate a single model
def evaluate_model(model, train, test, n_input, col):
	# history is a list of weekly data
	history = [x for x in train]
	# walk-forward validation over each week
	predictions = list()
	for i in range(len(test)):
		# predict the week
		yhat_sequence = sklearn_predict(model, history, n_input,col)
		# store the predictions
		predictions.append(yhat_sequence)
		# get real observation and add to history for predicting the next week
		history.append(test[i, :])
	predictions = array(predictions)
	# evaluate predictions days for each week
	score, scores , predicted= evaluate_forecasts(test[:, :, col], predictions)
	return score, scores , predicted

In [69]:
op=pd.DataFrame(columns=['PRN', 'Epoch_Time_of_Clock', 'SV_Clock_Bias', 'SV_Clock_Drift',
       'SV_Clock_Drift_Rate', 'IODE', 'Crs', 'Del_n', 'M0', 'Cuc', 'e', 'Cus',
       'sqrt_A', 'Toe', 'Cic', 'OMEGA', 'Cis', 'i0', 'Crc', 'omega',
       'OMEGA_dot', 'I_dot', 'Codes', 'GPS_week', 'L2_P_Data_flag',
       'SV_accuracy', 'SV_health', 'Tgd', 'IODC', 'T_Tx', 'Fit_Interval'])
op.shape

(0, 31)

In [70]:
op1=pd.DataFrame(columns=['PRN', 'Epoch_Time_of_Clock', 'SV_Clock_Bias', 'SV_Clock_Drift',
       'SV_Clock_Drift_Rate', 'IODE', 'Crs', 'Del_n', 'M0', 'Cuc', 'e', 'Cus',
       'sqrt_A', 'Toe', 'Cic', 'OMEGA', 'Cis', 'i0', 'Crc', 'omega',
       'OMEGA_dot', 'I_dot', 'Codes', 'GPS_week', 'L2_P_Data_flag',
       'SV_accuracy', 'SV_health', 'Tgd', 'IODC', 'T_Tx', 'Fit_Interval'])
dataset = pd.read_csv('1.csv')
# split into train and test
data=dataset.values
n_input = 6
train, test = data[0:168], data[168:174]   
# restructure into windows of weekly data
train = array(split(train, len(train)/n_input))
test = array(split(test, len(test)/n_input))
#train, test = split_dataset(dataset.values)
#print(test[6])
# prepare the models to evaluate

# evaluate each model
for i in range(4,33):
    model = get_models(i)
    print(model)
    print(i)
    # evaluate and get scores
    score, scores ,predicted  = evaluate_model(model, train, test, n_input, i)
    # summarize scores
    print("Score " , score)
    print("Predicted" , predicted)
    op1.iloc[:,i-2]= predicted
    for j in range (n_input):
        #print(predicted[j])
        op1.iloc[j,0]="1"
       # op.iloc[j,1]=dataset[]
    #summarize_scores(name, score, scores)
    # plot scores
    #pyplot.plot(days, scores, marker='o', label=name)
# show plot
#pyplot.legend()
#pyplot.show()
j=171
for i in range (n_input):
    op1.iloc[i,1]=dataset.iloc[j,3]
    j=j+1

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)
4
Score  5.003872733657553e-08
Predicted [-1.32790629e-05 -1.32957818e-05 -1.33134082e-05 -1.33311105e-05
 -1.34431905e-05 -1.34722849e-05]
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)
5
Score  6.077431683935706e-15
Predicted [-2.49924739e-12 -2.49740333e-12 -2.49558542e-12 -2.49379333e-12
 -2.49264770e-12 -2.49152082e-12]
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)
6
Score  0.0
Predicted [0. 0. 0. 0. 0. 0.]
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)
7
Score  31.987181019847156
Predicted [80.47250305 74.0314465  70.47869097 68.9570467  68.75468025 69.31691475]
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)
8
Score  19.468252944708293
Predicted [-14.93542638 -15.36017105 -17.30600397 -13.19400947 -12.57543579
 -13.37182378]
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, n

/home/shreyas/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/shreyas/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/shreyas/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/shreyas/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/shreyas/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWa

Score  0.3806969751587104
Predicted [0.66807936 0.79980268 0.17415652 0.01748343 0.0258661  0.06504752]
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)
28
Score  0.0
Predicted [0. 0. 0. 0. 0. 0.]
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)
29
Score  0.0
Predicted [5.58793545e-09 5.58793545e-09 5.58793545e-09 5.58793545e-09
 5.58793545e-09 5.58793545e-09]
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)
30
Score  31.987181019847156
Predicted [80.47250305 74.0314465  70.47869097 68.9570467  68.75468025 69.31691475]
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)
31
Score  11546.97571885141
Predicted [239181.88772178 259653.25214311 277577.16438919 293299.1299246
 303455.57297059 312067.10701304]
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)
32
Score  0.0
Predicted [0. 0. 0. 0. 0. 0.]


/home/shreyas/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/shreyas/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/shreyas/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/shreyas/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/shreyas/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWar

In [14]:
op3=pd.DataFrame(columns=['PRN', 'Epoch_Time_of_Clock', 'SV_Clock_Bias', 'SV_Clock_Drift',
       'SV_Clock_Drift_Rate', 'IODE', 'Crs', 'Del_n', 'M0', 'Cuc', 'e', 'Cus',
       'sqrt_A', 'Toe', 'Cic', 'OMEGA', 'Cis', 'i0', 'Crc', 'omega',
       'OMEGA_dot', 'I_dot', 'Codes', 'GPS_week', 'L2_P_Data_flag',
       'SV_accuracy', 'SV_health', 'Tgd', 'IODC', 'T_Tx', 'Fit_Interval'])
dataset = pd.read_csv('3.csv')
# split into train and test
data=dataset.values
n_input = 5
train, test = data[0:168], data[168:174]   
# restructure into windows of weekly data
train = array(split(train, len(train)/n_input))
test = array(split(test, len(test)/n_input))
#train, test = split_dataset(dataset.values)
#print(test[6])
# prepare the models to evaluate

# evaluate each model
for i in range(4,33):
    models = get_models(i)
    #print(models)
    #print(i)
    # evaluate and get scores
    score, scores ,predicted  = evaluate_model(model, train, test, n_input, i)
    # summarize scores
    print("Score " , score)
    print("Predicted" , predicted)
    op1.iloc[:,i-2]= predicted
    for j in range (n_input):
        #print(predicted[j])
        op1.iloc[j,0]="1"
       # op.iloc[j,1]=dataset[]
    summarize_scores(name, score, scores)
    # plot scores
    #pyplot.plot(days, scores, marker='o', label=name)
# show plot
#pyplot.legend()
#pyplot.show()
j=171
for i in range (n_input):
    op1.iloc[i,1]=dataset.iloc[j,3]
    j=j+1
#op1

FileNotFoundError: File b'3.csv' does not exist

In [144]:
dataset = pd.read_csv('2.csv')
# split into train and test
train, test = split_dataset(dataset.values)
#print(test[6])
# prepare the models to evaluate
models = get_models()
n_input = 
# evaluate each model
for i in range(4,33):
    for name, model in models.items():
        # evaluate and get scores
        score, scores ,predicted  = evaluate_model(model, train, test, n_input, i)
        # summarize scores
        print("Score " , score)
        print("Predicted" , predicted)
        op.iloc[:,i-2]= predicted
        for j in range (6):
            #print(predicted[j])
            op.iloc[j,0]="1"
           # op.iloc[j,1]=dataset[]
        summarize_scores(name, score, scores)
    # plot scores
    #pyplot.plot(days, scores, marker='o', label=name)
# show plot
#pyplot.legend()
#pyplot.show()
j=171
for i in range (6):
    op.iloc[i,1]=dataset.iloc[j,3]
    j=j+1


In [ ]:
dataset = pd.read_csv('3.csv')
# split into train and test
train, test = split_dataset(dataset.values)
#print(test[6])
# prepare the models to evaluate
models = get_models()
n_input = 6
# evaluate each model
for i in range(4,33):
    for name, model in models.items():
        # evaluate and get scores
        score, scores ,predicted  = evaluate_model(model, train, test, n_input, i)
        # summarize scores
        print("Score " , score)
        print("Predicted" , predicted)
        op.iloc[:,i-2]= predicted
        for j in range (6):
            #print(predicted[j])
            op.iloc[j,0]="1"
           # op.iloc[j,1]=dataset[]
        summarize_scores(name, score, scores)
    # plot scores
    #pyplot.plot(days, scores, marker='o', label=name)
# show plot
#pyplot.legend()
#pyplot.show()
j=171
for i in range (6):
    op.iloc[i,1]=dataset.iloc[j,3]
    j=j+1


In [145]:
op

,PRN,Epoch_Time_of_Clock,SV_Clock_Bias,SV_Clock_Drift,SV_Clock_Drift_Rate,IODE,Crs,Del_n,M0,Cuc,...,I_dot,Codes,GPS_week,L2_P_Data_flag,SV_accuracy,SV_health,Tgd,IODC,T_Tx,Fit_Interval
0,1,2017-11-29 12:00:00,-1.3279062881557141825e-05,-2.4992473910935048062e-12,0.0,80.472503051075165104,-14.935426377530745867,4.6808163107008418802e-09,-1.531560584992006202,-8.6971730379692342342e-07,...,2.585017774958139953e-10,1.0,"1,977.0129614702561867",0.0,0.66807936464850969926,0.0,5.5879354476899999498e-09,80.472503051075165104,"239,181.88772177748615",0.0
1,1,2017-11-29 14:00:00,-1.3295781750419825853e-05,-2.4974033292994470723e-12,0.0,74.03144650267044824,-15.360171051416529053,4.73614578180617766e-09,-0.64675930370704415306,-8.4196521926123754344e-07,...,1.8695266591382213853e-10,1.0,"1,977.0255209113527144",0.0,0.79980267902013524317,0.0,5.5879354476899999498e-09,74.03144650267044824,"259,653.25214311131276",0.0
2,1,2017-11-29 16:00:00,-1.3313408204265953088e-05,-2.4955854224258410517e-12,0.0,70.478690965293253612,-17.306003967010141764,4.7090142824247915561e-09,0.40837669638651319381,-9.2718006158153447126e-07,...,1.5233345123936754889e-10,1.0,"1,977.0376907931286041",0.0,0.17415652364375217021,0.0,5.5879354476899999498e-09,70.478690965293253612,"277,577.16438919096254",0.0
3,1,2017-11-29 18:00:00,-1.3331110531203020582e-05,-2.4937933289739060409e-12,0.0,68.957046701384058451,-13.194009467135217051,4.6586163531424217813e-09,1.595666554012381555,-7.522495698695486722e-07,...,1.5043253112318178354e-10,1.0,"1,977.0494831986422923",0.0,0.017483434423655214163,0.0,5.5879354476899999498e-09,68.957046701384058451,"293,299.12992460234091",0.0
4,1,2017-11-29 19:59:44,-1.3443190507513344179e-05,-2.4926476954522964706e-12,0.0,68.754680248556923061,-12.575435785140484768,4.6629210321195461988e-09,2.2977065874841837712,-7.3924082555790399037e-07,...,2.1207679442422526743e-10,1.0,"1,977.0609098361694578",0.0,0.025866096824145284883,0.0,5.5879354476899999498e-09,68.754680248556923061,"303,455.57297058589756",0.0
5,1,2017-11-29 22:00:00,-1.3472284901521261458e-05,-2.4915208219313900897e-12,0.0,69.316914750632747655,-13.371823775078766516,4.6713358016583858992e-09,-1.928477910205575796,-7.9360997448979117249e-07,...,2.4159330653689416355e-10,1.0,"1,977.0719820508270459",0.0,0.065047518310915791728,0.0,5.5879354476899999498e-09,69.316914750632747655,"312,067.10701304464601",0.0


In [141]:
dataset

,Unnamed: 0,Unnamed: 0.1,PRN,Epoch_Time_of_Clock,SV_Clock_Bias,SV_Clock_Drift,SV_Clock_Drift_Rate,IODE,Crs,Del_n,...,I_dot,Codes,GPS_week,L2_P_Data_flag,SV_accuracy,SV_health,Tgd,IODC,T_Tx,Fit_Interval
0,0,80,1,2017-11-01 14:00:00,-7.6745636761199992847e-06,-2.3874235921499998426e-12,0,30,20.71875,4.874845914039999901e-09,...,2.2393789934099997288e-10,1,1973,0,0,0,5.5879354476899999498e-09,30,302466,0
1,1,88,1,2017-11-01 16:00:00,-7.6913274824599997802e-06,-2.3874235921499998426e-12,0,31,16.8125,4.9169905266799998508e-09,...,3.0465554726900001994e-10,1,1973,0,0,0,5.5879354476899999498e-09,31,309606,0
2,2,101,1,2017-11-01 18:00:00,-7.7085569500899998858e-06,-2.3874235921499998426e-12,0,32,13.84375,4.874845914039999901e-09,...,2.971552348510000147e-10,1,1973,0,0,0,5.5879354476899999498e-09,32,316806,0
3,3,109,1,2017-11-01 20:00:00,-7.7253207564399987582e-06,-2.3874235921499998426e-12,0,33,12.28125,4.8752030717799997284e-09,...,2.5822504182400002217e-10,1,1973,0,0,0,5.5879354476899999498e-09,33,324006,0
4,4,124,1,2017-11-01 22:00:00,-7.7420845627800026419e-06,-2.3874235921499998426e-12,0,34,23.03125,4.7530551266799999166e-09,...,2.0000833115000001666e-10,1,1973,0,0,0,5.5879354476899999498e-09,34,331206,0
5,5,135,1,2017-11-02 00:00:00,-7.7593140304100010533e-06,-2.3874235921499998426e-12,0,69,29.0,4.7416260791900005539e-09,...,2.5465346448199998183e-10,1,1973,0,1,0,5.5879354476899999498e-09,69,338406,0
6,6,218,1,2017-11-02 12:00:00,-7.8575685620300011076e-06,-2.3874235921499998426e-12,0,73,40.03125,4.9194906308200003243e-09,...,3.4822879084099999123e-10,1,1973,0,0,0,5.5879354476899999498e-09,73,387756,0
7,7,221,1,2017-11-02 14:00:00,-7.874332368369999909e-06,-2.3874235921499998426e-12,0,74,46.5625,4.8519878190499997346e-09,...,4.1644591807300001016e-10,1,1973,0,0,0,5.5879354476899999498e-09,74,388806,0
8,8,233,1,2017-11-02 16:00:00,-7.8910961747200004756e-06,-2.3874235921499998426e-12,0,75,43.84375,4.8905608543500001287e-09,...,5.128785063059999523e-10,1,1973,0,0,0,5.5879354476899999498e-09,75,396006,0
9,9,244,1,2017-11-02 18:00:00,-7.9078599810600009711e-06,-2.3874235921499998426e-12,0,76,41.75,4.8662741284199993983e-09,...,4.9680640826699995171e-10,1,1973,0,0,0,5.5879354476899999498e-09,76,403206,0


In [143]:
data = pd.read_csv('1.csv')
data=data.values
print(data.shape)
train, test = data[0:140], data[140:175]
# restructure into windows of weekly data
train = array(split(train, len(train)/5))
test = array(split(test, len(test)/5))


(177, 33)


In [154]:
print(train[0])

[[0 80 1 '2017-11-01 14:00:00' -7.67456367612e-06 -2.38742359215e-12 0 30
  20.71875 4.87484591404e-09 2.75563657825 1.01327896118e-06
  0.007094035274350001 3.87243926525e-06 5153.67019653 309600
  -8.94069671631e-08 1.0093728597 -6.14672899246e-08 0.968711908956 309.375
  0.63272009924 -8.2667729157e-09 2.2393789934099997e-10 1 1973 0 0 0
  5.58793544769e-09 30 302466 0]
 [1 88 1 '2017-11-01 16:00:00' -7.69132748246e-06 -2.38742359215e-12 0 31
  16.8125 4.91699052668e-09 -2.47738129409 7.264316082e-07 0.00709456193727
  3.84263694286e-06 5153.66918755 316800 3.35276126862e-08
  1.0093125596799999 -1.32247805595e-07 0.9687134757420001 310.125
  0.632738922607 -8.350704983239999e-09 3.04655547269e-10 1 1973 0 0 0
  5.58793544769e-09 31 309606 0]
 [2 101 1 '2017-11-01 18:00:00' -7.70855695009e-06 -2.38742359215e-12 0 32
  13.84375 4.87484591404e-09 -1.42713945816 7.022172212600001e-07
  0.0070945401676000015 3.7644058466e-06 5153.66988754 324000
  1.15483999252e-07 1.00925244691 -8.3819

In [93]:
dataset.head()

,Unnamed: 0,Unnamed: 0.1,PRN,Epoch_Time_of_Clock,SV_Clock_Bias,SV_Clock_Drift,SV_Clock_Drift_Rate,IODE,Crs,Del_n,...,I_dot,Codes,GPS_week,L2_P_Data_flag,SV_accuracy,SV_health,Tgd,IODC,T_Tx,Fit_Interval
0,0,80,1,2017-11-01 14:00:00,-7.6745636761199992847e-06,-2.3874235921499998426e-12,0,30,20.71875,4.874845914039999901e-09,...,2.2393789934099997288e-10,1,1973,0,0,0,5.5879354476899999498e-09,30,302466,0
1,1,88,1,2017-11-01 16:00:00,-7.6913274824599997802e-06,-2.3874235921499998426e-12,0,31,16.8125,4.9169905266799998508e-09,...,3.0465554726900001994e-10,1,1973,0,0,0,5.5879354476899999498e-09,31,309606,0
2,2,101,1,2017-11-01 18:00:00,-7.7085569500899998858e-06,-2.3874235921499998426e-12,0,32,13.84375,4.874845914039999901e-09,...,2.971552348510000147e-10,1,1973,0,0,0,5.5879354476899999498e-09,32,316806,0
3,3,109,1,2017-11-01 20:00:00,-7.7253207564399987582e-06,-2.3874235921499998426e-12,0,33,12.28125,4.8752030717799997284e-09,...,2.5822504182400002217e-10,1,1973,0,0,0,5.5879354476899999498e-09,33,324006,0
4,4,124,1,2017-11-01 22:00:00,-7.7420845627800026419e-06,-2.3874235921499998426e-12,0,34,23.03125,4.7530551266799999166e-09,...,2.0000833115000001666e-10,1,1973,0,0,0,5.5879354476899999498e-09,34,331206,0


In [94]:
df = pd.read_csv('1.csv')

In [95]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'PRN', 'Epoch_Time_of_Clock',
       'SV_Clock_Bias', 'SV_Clock_Drift', 'SV_Clock_Drift_Rate', 'IODE', 'Crs',
       'Del_n', 'M0', 'Cuc', 'e', 'Cus', 'sqrt_A', 'Toe', 'Cic', 'OMEGA',
       'Cis', 'i0', 'Crc', 'omega', 'OMEGA_dot', 'I_dot', 'Codes', 'GPS_week',
       'L2_P_Data_flag', 'SV_accuracy', 'SV_health', 'Tgd', 'IODC', 'T_Tx',
       'Fit_Interval'],
      dtype='object')

In [96]:
def split_dataset(data):
	# split into standard weeks
	train, test = data[5:140], data[140:175]
	# restructure into windows of weekly data
	train = array(split(train, len(train)/5))
	test = array(split(test, len(test)/5))
	return train, test

In [97]:
train, test = split_dataset(df.values)

In [98]:
train.shape

(27, 5, 33)

In [99]:
train[0]

array([[5, 135, 1, '2017-11-02 00:00:00', -7.759314030410001e-06,
        -2.38742359215e-12, 0, 69, 29.0, 4.7416260791900006e-09,
        1.72347762985, 1.42678618431e-06, 0.00709622702561,
        3.90969216824e-06, 5153.67224121, 345600, -1.210719347e-07,
        1.00907040139, 8.56816768646e-08, 0.9687207186489999, 308.28125,
        0.632618838977, -8.38927801853e-09, 2.54653464482e-10, 1, 1973, 0,
        1, 0, 5.58793544769e-09, 69, 338406, 0],
       [6, 218, 1, '2017-11-02 12:00:00', -7.857568562030001e-06,
        -2.38742359215e-12, 0, 73, 40.03125, 4.91949063082e-09,
        1.7415583478799999, 2.08988785744e-06, 0.00709870201536,
        4.51132655144e-06, 5153.67212868, 388800, -1.97440385818e-07,
        1.00870849593, 1.4528632164e-07, 0.968731389174, 297.03125,
        0.632462477904, -8.39749264642e-09, 3.48228790841e-10, 1, 1973, 0,
        0, 0, 5.58793544769e-09, 73, 387756, 0],
       [7, 221, 1, '2017-11-02 14:00:00', -7.87433236837e-06,
        -2.38742359215e-1

In [100]:
models = get_models()

Defined 1 models


In [101]:
models['lr']

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [102]:
hist = [x for x in train]
len(hist)

27

In [103]:
hist[0].shape

(5, 33)

In [104]:
train_x , train_y = to_supervised(hist , 5)

In [105]:
#train_x=np.square(train_x)

In [106]:
train_y[0]

5153.6695938100002

In [107]:
pipeline = make_pipeline(models['lr'])

In [108]:
pipeline.fit(train_x,train_y)

/usr/lib/python3/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Pipeline(memory=None,
     steps=[('standardize', StandardScaler(copy=True, with_mean=True, with_std=True)), ('normalize', MinMaxScaler(copy=True, feature_range=(0, 1))), ('model', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False))])

In [109]:
input_x = train_x[-1, : ]
input_x

array([5153.68056679, 5153.68127251, 5153.68191147, 5153.68182182,
       5153.68195343], dtype=object)

In [110]:
input_data = [x for x in input_x]
input_data

[5153.68056679, 5153.68127251, 5153.68191147, 5153.68182182, 5153.68195343]

In [111]:
yhat_sequence = []
for j in range(5):
    # prepare the input data
    X = array(input_data[-n_input:]).reshape(1, n_input)
    print(X)
    # make a one-step forecast
    yhat = model.predict(X)[0]
    print("**")
    print(yhat)
    # add to the result
    yhat_sequence.append(yhat)
    # add the prediction to the input
    input_data.append(yhat)

[[ 5153.68056679  5153.68127251  5153.68191147  5153.68182182
   5153.68195343]]
**
5233.79431047
[[ 5153.68127251  5153.68191147  5153.68182182  5153.68195343
   5233.79431047]]
**
5234.51691533
[[ 5153.68191147  5153.68182182  5153.68195343  5233.79431047
   5234.51691533]]
**
5235.02615356
[[ 5153.68182182  5153.68195343  5233.79431047  5234.51691533
   5235.02615356]]
**
5234.61253303
[[ 5153.68195343  5233.79431047  5234.51691533  5235.02615356
   5234.61253303]]
**
5235.07071947


In [112]:
test.shape

(7, 5, 33)

In [113]:
# history is a list of weekly data
history = [x for x in train]
# walk-forward validation over each week
predictions = list()
for i in range(len(test)):
    # predict the week
    yhat_sequence = sklearn_predict(model, history, n_input)
    # store the predictions
    predictions.append(yhat_sequence)
    # get real observation and add to history for predicting the next week
    history.append(test[i, :])
predictions = array(predictions)

/usr/lib/python3/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/lib/python3/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/lib/python3/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/lib/python3/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/lib/python3/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, Dat

In [114]:
predictions

array([[ 5153.68164822,  5153.68177317,  5153.68161354,  5153.68172107,
         5153.68155912],
       [ 5153.68070318,  5153.68011822,  5153.68079459,  5153.68027835,
         5153.68064454],
       [ 5153.67928056,  5153.67901454,  5153.67922136,  5153.67898044,
         5153.67923425],
       [ 5153.6772644 ,  5153.67682071,  5153.67750389,  5153.67695566,
         5153.67747976],
       [ 5153.67560733,  5153.67680053,  5153.67658464,  5153.67667151,
         5153.67640365],
       [ 5153.67612098,  5153.67685544,  5153.67587238,  5153.67701038,
         5153.67635401],
       [ 5153.67337766,  5153.67324566,  5153.6733438 ,  5153.67350885,
         5153.67376678]])

In [115]:
score, scores = evaluate_forecasts(test[:, :, 14], predictions)

Printing actual and predicted  [5153.68014526 5153.68021965 5153.68157387 5153.68034363 5153.67912865] [ 5153.68164822  5153.68177317  5153.68161354  5153.68172107  5153.68155912]


ValueError: too many values to unpack (expected 2)

In [ ]:
score

In [ ]:
scores

In [ ]:
test[:, :, 14]

In [ ]:
dataset = pd.read_csv('1.csv')
# split into train and test
train, test = split_dataset(dataset.values)
# prepare the models to evaluate
models = get_models()
n_input = 5
# evaluate each model
days = ['one', 'two', 'three', 'four', 'five']
for name, model in models.items():
    # evaluate and get scores
    score, scores = evaluate_model(model, train, test, n_input)
    # summarize scores
    print("Score " , score)
    summarize_scores(name, score, scores)
    # plot scores
    pyplot.plot(days, scores, marker='o', label=name)
# show plot
pyplot.legend()
pyplot.show()

In [ ]:
len(score)